Scraping amazon reviews from the Amazon Website

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline

In [2]:
search_query = 'pendrive&ref=nb_sb_noss_2'

In [3]:
base_url = 'https://www.amazon.in/s?k='

In [4]:
url = base_url+search_query

In [5]:
url

'https://www.amazon.in/s?k=pendrive&ref=nb_sb_noss_2'

In [6]:
header={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'}

In [7]:
search_response=requests.get(url,headers=header)

In [8]:
search_response.status_code

200

In [9]:
search_response.text

'<!doctype html><html lang="en-in" class="a-no-js" data-19ax5a9jf="dingo"><!-- sp:feature:head-start -->\n<head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/>\n\n\n\n<script>var ue_t0=ue_t0||+new Date();</script>\n\n<!-- sp:feature:cs-optimization -->\n<meta http-equiv=\'x-dns-prefetch-control\' content=\'on\'>\n<link rel="dns-prefetch" href="https://images-eu.ssl-images-amazon.com">\n<link rel="dns-prefetch" href="https://m.media-amazon.com">\n<link rel="dns-prefetch" href="https://completion.amazon.com">\n\n<script>\n(function(a){var b=a.performance;a.ue_mark=b&&"function"===typeof b.mark?function(a){b.mark(a)}:function(){}})(window);\n\n    ue_mark(\'csm:ho:ob\');\n</script>\n<script>\nwindow.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;\n\nif (window.ue_ihb === 1) {\n\n    var ue_csm = window,\n        ue_hob = +new Date();\n(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.st

Function to get content of the page of required query

In [10]:
cookie={} # insert request cookies within{}
def getAmazonSearch(search_query):
    url="https://www.amazon.in/s?k="+search_query
    print(url)
    page=requests.get(url,cookies=cookie,headers=header)
    if page.status_code==200:
        return page
    else:
        return "Error"

Function to get the contents of individual product pages using 'data-asin' number (unique identification Number)

In [12]:
def Searchasin(asin):
    url="https://www.amazon.in/dp/"+asin
    print(url)
    page=requests.get(url,cookies=cookie,headers=header)
    if page.status_code==200:
        return page
    else:
        return "Error"


A function to pass on the link of 'see all reviews' and extract the content

In [13]:
def Searchreviews(review_link):
    url="https://www.amazon.in"+review_link
    print(url)
    page=requests.get(url,cookies=cookie,headers=header)
    if page.status_code==200:
        return page
    else:
        return "Error"

First page product reviews extraction

In [14]:
product_names=[]
response=getAmazonSearch('pendrive&ref=nb_sb_noss_2')
soup=BeautifulSoup(response.content)
for i in soup.findAll("span",{'class':'a-size-base-plus a-color-base a-text-normal'}): # the tag which is common for all the names of products
    product_names.append(i.text) #adding the product names to the list

https://www.amazon.in/s?k=pendrive&ref=nb_sb_noss_2



findall() is used to find all the tags with the mentioned attributes and values in a page

In [15]:
product_names

['Toshiba THN-U202W0320E4 USB Flash Drive',
 'SanDisk 32GB Ultra Fit USB 3.1 Flash Drive - SDCZ430-032G-G46 (16 GB)',
 'Kingston DataTraveler DT100G3 32GB USB 3.0 Flash Drive (Black)',
 'SanDisk Ultra Flair 64GB USB 3.0 Pen Drive',
 'SanDisk Ultra Flair 128GB USB 3.0 Pen Drive',
 'SanDisk Ultra Dual SDDD3-128G-I35 USB 3.0 128GB Flash Drive',
 'SanDisk Ultra Flair 128GB USB 3.0 Pen Drive',
 'Lexar Professional 1667x 128GB SDXC UHS-II/U3 Card (LSD128CBNA1667)',
 'Samsung EVO Plus 128GB microSDXC UHS-I U3 100MB/s Full HD & 4K UHD Memory Card with Adapter (MB-MC128GA)']

The method of extracting data-asin numbers are similar to that of product names. Only the tag details have to be changed in findall()

In [17]:
data_asin=[]
response=getAmazonSearch('pendrive&ref=nb_sb_noss_2')
soup=BeautifulSoup(response.content)
for i in soup.findAll("div",{'class':"sg-col-4-of-24 sg-col-4-of-12 sg-col-4-of-36 s-result-item sg-col-4-of-28 sg-col-4-of-16 sg-col sg-col-4-of-20 sg-col-4-of-32"}):
    data_asin.append(i['data-asin'])

https://www.amazon.in/s?k=pendrive&ref=nb_sb_noss_2


In [18]:
data_asin

['B014VM3G3U', 'B077Y149DL', 'B00C5K8E1A', 'B015CH1NAQ', 'B015CH1PJU']

By passing the data-asin numbers, we can extract the 'see all reviews' link for each product in the page

In [19]:
link=[]
for i in range(len(data_asin)):
    response=Searchasin(data_asin[i])
    soup=BeautifulSoup(response.content)
    for i in soup.findAll("a",{'data-hook':"see-all-reviews-link-foot"}):
        link.append(i['href'])

https://www.amazon.in/dp/B014VM3G3U
https://www.amazon.in/dp/B077Y149DL
https://www.amazon.in/dp/B00C5K8E1A
https://www.amazon.in/dp/B015CH1NAQ
https://www.amazon.in/dp/B015CH1PJU



Now we have the 'see all review' links. Using this link along with a page number, we can extract the reviews in any number of pages for all the products

In [20]:
reviews=[]
for j in range(len(link)):
    for k in range(100):
        response=Searchreviews(link[j]+'&pageNumber='+str(k))
        soup=BeautifulSoup(response.content)
        for i in soup.findAll("span",{'data-hook':"review-body"}):
            reviews.append(i.text)

https://www.amazon.in/Toshiba-THN-U202W0320E4-USB-Flash-Drive/product-reviews/B014VM3G3U/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=0
https://www.amazon.in/Toshiba-THN-U202W0320E4-USB-Flash-Drive/product-reviews/B014VM3G3U/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
https://www.amazon.in/Toshiba-THN-U202W0320E4-USB-Flash-Drive/product-reviews/B014VM3G3U/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=2
https://www.amazon.in/Toshiba-THN-U202W0320E4-USB-Flash-Drive/product-reviews/B014VM3G3U/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=3
https://www.amazon.in/Toshiba-THN-U202W0320E4-USB-Flash-Drive/product-reviews/B014VM3G3U/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=4
https://www.amazon.in/Toshiba-THN-U202W0320E4-USB-Flash-Drive/product-reviews/B014VM3G3U/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=5
https://www.amazon.in/

https://www.amazon.in/Toshiba-THN-U202W0320E4-USB-Flash-Drive/product-reviews/B014VM3G3U/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=51
https://www.amazon.in/Toshiba-THN-U202W0320E4-USB-Flash-Drive/product-reviews/B014VM3G3U/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=52
https://www.amazon.in/Toshiba-THN-U202W0320E4-USB-Flash-Drive/product-reviews/B014VM3G3U/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=53
https://www.amazon.in/Toshiba-THN-U202W0320E4-USB-Flash-Drive/product-reviews/B014VM3G3U/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=54
https://www.amazon.in/Toshiba-THN-U202W0320E4-USB-Flash-Drive/product-reviews/B014VM3G3U/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=55
https://www.amazon.in/Toshiba-THN-U202W0320E4-USB-Flash-Drive/product-reviews/B014VM3G3U/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=56
https://www.amaz

https://www.amazon.in/Toshiba-THN-U202W0320E4-USB-Flash-Drive/product-reviews/B014VM3G3U/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
https://www.amazon.in/Toshiba-THN-U202W0320E4-USB-Flash-Drive/product-reviews/B014VM3G3U/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=2
https://www.amazon.in/Toshiba-THN-U202W0320E4-USB-Flash-Drive/product-reviews/B014VM3G3U/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=3
https://www.amazon.in/Toshiba-THN-U202W0320E4-USB-Flash-Drive/product-reviews/B014VM3G3U/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=4
https://www.amazon.in/Toshiba-THN-U202W0320E4-USB-Flash-Drive/product-reviews/B014VM3G3U/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=5
https://www.amazon.in/Toshiba-THN-U202W0320E4-USB-Flash-Drive/product-reviews/B014VM3G3U/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=6
https://www.amazon.in/

https://www.amazon.in/Toshiba-THN-U202W0320E4-USB-Flash-Drive/product-reviews/B014VM3G3U/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=52
https://www.amazon.in/Toshiba-THN-U202W0320E4-USB-Flash-Drive/product-reviews/B014VM3G3U/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=53
https://www.amazon.in/Toshiba-THN-U202W0320E4-USB-Flash-Drive/product-reviews/B014VM3G3U/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=54
https://www.amazon.in/Toshiba-THN-U202W0320E4-USB-Flash-Drive/product-reviews/B014VM3G3U/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=55
https://www.amazon.in/Toshiba-THN-U202W0320E4-USB-Flash-Drive/product-reviews/B014VM3G3U/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=56
https://www.amazon.in/Toshiba-THN-U202W0320E4-USB-Flash-Drive/product-reviews/B014VM3G3U/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=57
https://www.amaz

https://www.amazon.in/SanDisk-32GB-Ultra-Flash-Drive/product-reviews/B077Y149DL/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=3
https://www.amazon.in/SanDisk-32GB-Ultra-Flash-Drive/product-reviews/B077Y149DL/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=4
https://www.amazon.in/SanDisk-32GB-Ultra-Flash-Drive/product-reviews/B077Y149DL/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=5
https://www.amazon.in/SanDisk-32GB-Ultra-Flash-Drive/product-reviews/B077Y149DL/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=6
https://www.amazon.in/SanDisk-32GB-Ultra-Flash-Drive/product-reviews/B077Y149DL/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=7
https://www.amazon.in/SanDisk-32GB-Ultra-Flash-Drive/product-reviews/B077Y149DL/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=8
https://www.amazon.in/SanDisk-32GB-Ultra-Flash-Drive/product-reviews/B077Y14

...

https://www.amazon.in/SanDisk-Ultra-Flair-64GB-Drive/product-reviews/B015CH1NAQ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=75
https://www.amazon.in/SanDisk-Ultra-Flair-64GB-Drive/product-reviews/B015CH1NAQ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=76
https://www.amazon.in/SanDisk-Ultra-Flair-64GB-Drive/product-reviews/B015CH1NAQ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=77
https://www.amazon.in/SanDisk-Ultra-Flair-64GB-Drive/product-reviews/B015CH1NAQ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=78
https://www.amazon.in/SanDisk-Ultra-Flair-64GB-Drive/product-reviews/B015CH1NAQ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=79
https://www.amazon.in/SanDisk-Ultra-Flair-64GB-Drive/product-reviews/B015CH1NAQ/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=80
https://www.amazon.in/SanDisk-Ultra-Flair-64GB-Drive/product-reviews/B

https://www.amazon.in/SanDisk-Ultra-Flair-128GB-Drive/product-reviews/B015CH1PJU/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=28
https://www.amazon.in/SanDisk-Ultra-Flair-128GB-Drive/product-reviews/B015CH1PJU/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=29
https://www.amazon.in/SanDisk-Ultra-Flair-128GB-Drive/product-reviews/B015CH1PJU/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=30
https://www.amazon.in/SanDisk-Ultra-Flair-128GB-Drive/product-reviews/B015CH1PJU/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=31
https://www.amazon.in/SanDisk-Ultra-Flair-128GB-Drive/product-reviews/B015CH1PJU/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=32
https://www.amazon.in/SanDisk-Ultra-Flair-128GB-Drive/product-reviews/B015CH1PJU/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=33
https://www.amazon.in/SanDisk-Ultra-Flair-128GB-Drive/product-re

https://www.amazon.in/SanDisk-Ultra-Flair-128GB-Drive/product-reviews/B015CH1PJU/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=81
https://www.amazon.in/SanDisk-Ultra-Flair-128GB-Drive/product-reviews/B015CH1PJU/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=82
https://www.amazon.in/SanDisk-Ultra-Flair-128GB-Drive/product-reviews/B015CH1PJU/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=83
https://www.amazon.in/SanDisk-Ultra-Flair-128GB-Drive/product-reviews/B015CH1PJU/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=84
https://www.amazon.in/SanDisk-Ultra-Flair-128GB-Drive/product-reviews/B015CH1PJU/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=85
https://www.amazon.in/SanDisk-Ultra-Flair-128GB-Drive/product-reviews/B015CH1PJU/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=86
https://www.amazon.in/SanDisk-Ultra-Flair-128GB-Drive/product-re

https://www.amazon.in/SanDisk-Ultra-Flair-128GB-Drive/product-reviews/B015CH1PJU/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=34
https://www.amazon.in/SanDisk-Ultra-Flair-128GB-Drive/product-reviews/B015CH1PJU/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=35
https://www.amazon.in/SanDisk-Ultra-Flair-128GB-Drive/product-reviews/B015CH1PJU/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=36
https://www.amazon.in/SanDisk-Ultra-Flair-128GB-Drive/product-reviews/B015CH1PJU/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=37
https://www.amazon.in/SanDisk-Ultra-Flair-128GB-Drive/product-reviews/B015CH1PJU/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=38
https://www.amazon.in/SanDisk-Ultra-Flair-128GB-Drive/product-reviews/B015CH1PJU/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=39
https://www.amazon.in/SanDisk-Ultra-Flair-128GB-Drive/product-re

https://www.amazon.in/SanDisk-Ultra-Flair-128GB-Drive/product-reviews/B015CH1PJU/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=87
https://www.amazon.in/SanDisk-Ultra-Flair-128GB-Drive/product-reviews/B015CH1PJU/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=88
https://www.amazon.in/SanDisk-Ultra-Flair-128GB-Drive/product-reviews/B015CH1PJU/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=89
https://www.amazon.in/SanDisk-Ultra-Flair-128GB-Drive/product-reviews/B015CH1PJU/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=90
https://www.amazon.in/SanDisk-Ultra-Flair-128GB-Drive/product-reviews/B015CH1PJU/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=91
https://www.amazon.in/SanDisk-Ultra-Flair-128GB-Drive/product-reviews/B015CH1PJU/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=92
https://www.amazon.in/SanDisk-Ultra-Flair-128GB-Drive/product-re

In [21]:
len(reviews)

532

In [23]:
rev = {'reviews':reviews}#coverting into dictionary

In [24]:
review_data = pd.DataFrame.from_dict(rev)#converting into Dataframe

In [25]:
review_data.sample(5)

,reviews
437,\nThe recently procured 128 GB USB becomes qui...
327,\nThe Best Option in USB 3.0\n
377,\nMax read & write I got is 37.07 & 20.58 MB's...
244,\nA good option when compared to extreme 64gb ...
103,\nGood\n


In [26]:
review_data.shape

(532, 1)

In [29]:
review_data.head(10)

,reviews
0,\nI am using only Toshiba pen drive. This is m...
1,\nVery good memory card easy use\n
2,\nHighly reliable Japanese quality product. Us...
3,\nI am using only Toshiba pen drive. This is m...
4,\nVery good memory card easy use\n
5,\nHighly reliable Japanese quality product. Us...
6,\nI am using only Toshiba pen drive. This is m...
7,\nVery good memory card easy use\n
8,\nHighly reliable Japanese quality product. Us...
9,\nI am using only Toshiba pen drive. This is m...


In [30]:
import re
from bs4 import BeautifulSoup
import unicodedata
import spacy
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem import LancasterStemmer
ps = nltk.porter.PorterStemmer()
ls =nltk.stem.LancasterStemmer()
import requests 

import re
contractions_dict = {
    'didn\'t': 'did not',
    'don\'t': 'do not',
    "aren't": "are not",
    "can't": "cannot",
    "cant": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "didnt": "did not",
    "doesn't": "does not",
    "doesnt": "does not",
    "don't": "do not",
    "dont" : "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he had",
    "he'd've": "he would have",
    "he'll": "he will",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i had",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'm": "i am",
    "im": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'll": "it will",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she had",
    "she'd've": "she would have",
    "she'll": "she will",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "that's": "that is",
    "there's": "there is",
    "they'd": "they had",
    "they'd've": "they would have",
    "they'll": "they will",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we had",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who's": "who is",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "you'll": "you will",
    "you're": "you are",
    "you've": "you have"
    }

contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))

def expand_contractions(s, contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)



def strip_html_tags():
    soup = BeautifulSoup(content,"html.parser")
    [s.extract for s in soup(['iframe','script'])]
    stripped_text = soup.get_text() 
    stripped_text=re.sub('[\r|\n|\r\n]+','\n',stripped_text)
    return stripped_text

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD',text).encode('ascii','ignore').decode('utf-8','ignore')
    return text

def remove_special_characters(text, remove_digits = False):
    patterns = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern,"",text)
    return text

def simple_stemmers(text,stemmer = ps):
    text = " ".join([stemmer.stem(word)for word in text.split()])
    return text

def expand_contraction(text):
    return contraction.fix(text)

def spacy_lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ !='-PRON-' else word.text for word in text])

def remove_stopwords(text, is_lower_case = False, stopwords = None):
    if not stopwords:
        stopwords = nltk.corpus.stopwords.words('english')
    tokens = nltk.word_tokenize(text)
    tokens=[token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [tokens for token in tokens if token not in stopwords]
    else:
         filtered_tokens = [tokens for token in tokens if token.lower() not in stopwords]
    filtered_tokens = ' '.join(filtered_tokens)
    return filtered_tokens    

In [31]:
import tqdm

def text_pre_processor(text,html_strip=True, accented_char=True,contraction_expansion=True,text_lower_case=True,
                       text_stemming=False, text_lemmatization=True,special_char_removal=True, remove_digits=True,
                       stopword_removal=True, stopword_list=None): 
    #strip HTML
    if html_strip:
        text=strip_html_tags(text)
    
    #remove extra newlines(often might be present in really noisy text)
    text = text.translate(text.maketrans("\n\t\r"," "))

    #remove accented character
    if accented_char_removal:
        text = removal_accented_chars(text)
    
   #expand contraction
    if contraction_expansion:
         text = spacy_lemmatize_text(text)
    
   #Lemmatize text
    if text_lemmatization:
         text = spacy_lemmatize_text(text)
    
   #remove special characters and \or digits
    if special_char_removal:
   #insert space between special characters to isolate them
        special_char_pattern = re.compile(r'([{.(-)!}])')
        text = special_char_pattern.sub("\\1 ", text)
        text = remove_special_characters(text, remove_digits = remove_digits)
    
   #stem text
    if text_stemming and not text_lemmatization:
         text = simple_stemming(text)
    
   #lowercase the text
    if text_lower_case:
         text = text.lower()

   #remove stopwords
    if stopword_removal:
         text = remove_stopwords(text,is_lower_case = text_lower_case,stopwords=stopword_list)
    
   #remove extra whitespace
    text = re.sub(' +', ' ',text)
    text = text.strip()

    return text



In [32]:
from nltk.corpus import stopwords
import textblob
stop = stopwords.words('english')
from nltk.corpus import opinion_lexicon
pos_list=set(opinion_lexicon.positive())
neg_list=set(opinion_lexicon.negative())
from nltk.tokenize import treebank
tokenizer = treebank.TreebankWordTokenizer()
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import sentiwordnet as swn
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from afinn import Afinn

In [33]:
def corpus_pre_processor(corpus):
    norm_corpus = []
    for doc in tqdm.tqdm(corpus):
        norm_corpus.append(text_pre_processor(doc))
    return norm_corpus

def normalize_document(doc):
    #Lowercase, remove special char \whitespace
    #remove stopwords
    #expand contraction
    words= word_tokenize(doc)
    doc = " ".join([word.lower() for word in words if word not in stop])
    doc = re.sub(r'[^a-zA-Z0-9\s]','',doc,re.I|re.A)
    doc =doc.strip()
    doc = expand_contractions(doc)
    return doc

In [34]:
review_data.reset_index(inplace = True)

In [35]:
dataset = review_data

In [37]:
#split the review into sentence
data=dataset[['reviews','index']]
data.rename(columns ={'index':'INDEX'},inplace = True)

In [38]:
data.head()

,reviews,INDEX
0,\nI am using only Toshiba pen drive. This is m...,0
1,\nVery good memory card easy use\n,1
2,\nHighly reliable Japanese quality product. Us...,2
3,\nI am using only Toshiba pen drive. This is m...,3
4,\nVery good memory card easy use\n,4


In [39]:
from nltk.tokenize import sent_tokenize
data['split'] = data['reviews'].apply(sent_tokenize)

In [40]:
data.head()

,reviews,INDEX,split
0,\nI am using only Toshiba pen drive. This is m...,0,"[\nI am using only Toshiba pen drive., This is..."
1,\nVery good memory card easy use\n,1,[\nVery good memory card easy use]
2,\nHighly reliable Japanese quality product. Us...,2,"[\nHighly reliable Japanese quality product., ..."
3,\nI am using only Toshiba pen drive. This is m...,3,"[\nI am using only Toshiba pen drive., This is..."
4,\nVery good memory card easy use\n,4,[\nVery good memory card easy use]


In [41]:
data_split=data.set_index('INDEX').split.apply(pd.Series).stack().reset_index(level=0).rename(columns={0:'reviews'})
data_split.reset_index(level=0,inplace=True)
data_split.rename(columns={'INDEX':'review_no','index':'sentence'},inplace=True)

In [42]:
data_split.head(15)

,sentence,review_no,reviews
0,0,0,\nI am using only Toshiba pen drive.
1,1,0,This is my IIIrd puchasing from Amazon.
2,2,0,This device very easy for use
3,0,1,\nVery good memory card easy use
4,0,2,\nHighly reliable Japanese quality product.
5,1,2,Used it for a long time with no problem.
6,2,2,Data always safe and can survive rough use
7,0,3,\nI am using only Toshiba pen drive.
8,1,3,This is my IIIrd puchasing from Amazon.
9,2,3,This device very easy for use


In [43]:
#then normalizing the data
data_split['reviews']=data_split['reviews'].apply(normalize_document)

In [44]:
data_split.head()

,sentence,review_no,reviews
0,0,0,i using toshiba pen drive
1,1,0,this iiird puchasing amazon
2,2,0,this device easy use
3,0,1,very good memory card easy use
4,0,2,highly reliable japanese quality product


In [45]:
def score(reviews):
    from textblob import TextBlob
    return TextBlob(reviews).sentiment.polarity
def predict(reviews):
    data_split['score']=data_split['reviews'].apply(score)
    return(data_split)

In [46]:
data_split.head()

,sentence,review_no,reviews
0,0,0,i using toshiba pen drive
1,1,0,this iiird puchasing amazon
2,2,0,this device easy use
3,0,1,very good memory card easy use
4,0,2,highly reliable japanese quality product


In [47]:
sentiment_=predict(data_split)
sentiment_

,sentence,review_no,reviews,score
0,0,0,i using toshiba pen drive,0.000000
1,1,0,this iiird puchasing amazon,0.000000
2,2,0,this device easy use,0.433333
3,0,1,very good memory card easy use,0.671667
4,0,2,highly reliable japanese quality product,0.080000
...,...,...,...,...
905,0,529,damage products dai deaaaa returned date bhi k...,0.000000
906,0,530,it took 115 minutes copy 115gb movie,0.000000
907,1,530,received product shown i amage,0.000000
908,2,530,it genuine product,0.400000


Sentiment Analysis

In [48]:
import textblob

In [49]:
sentiment_['Sentiment']=['positive' if score >=0 else 'negative' for score in sentiment_['score']]

In [50]:
Reviews=np.array(sentiment_['reviews'])
sentiment =np.array(sentiment_['Sentiment'])
sample=[13,44,69]

In [51]:
Reviews

array(['i using toshiba pen drive', 'this iiird puchasing amazon',
       'this device easy use', 'very good memory card easy use',
       'highly reliable japanese quality product',
       'used long ti ame problem', 'data always safe survive rough use',
       'i using toshiba pen drive', 'this iiird puchasing amazon',
       'this device easy use', 'very good memory card easy use',
       'highly reliable japanese quality product',
       'used long ti ame problem', 'data always safe survive rough use',
       'i using toshiba pen drive', 'this iiird puchasing amazon',
       'this device easy use', 'very good memory card easy use',
       'highly reliable japanese quality product',
       'used long ti ame problem', 'data always safe survive rough use',
       'i using toshiba pen drive', 'this iiird puchasing amazon',
       'this device easy use', 'very good memory card easy use',
       'highly reliable japanese quality product',
       'used long ti ame problem', 'data always s

In [52]:
sentiment

array(['positive', 'positive', 'positive', 'positive', 'positive',
       'negative', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'negative', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'negative',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'negative', 'positive', 'positive', 'positive',
       'negative', 'positive', 'positive', 'negative', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'positive', 'negative', 'positive',
       'positive', 'negative', 'positive', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'negative', 'negative', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positi

In [53]:
for Reviews,sentiment in zip (Reviews[sample],sentiment[sample]):
    print('Reviews',Reviews)
    print('SENTIMENT',sentiment)
    print('Predicted Sentiment polarity',textblob.TextBlob(Reviews).sentiment.polarity)
    print('-'*60)

Reviews data always safe survive rough use
SENTIMENT positive
Predicted Sentiment polarity 0.2
------------------------------------------------------------
Reviews very small size  best car steario
SENTIMENT positive
Predicted Sentiment polarity 0.3375
------------------------------------------------------------
Reviews rigidity  durability  measures heat dissipation area concern drive4
SENTIMENT positive
Predicted Sentiment polarity 0.0
------------------------------------------------------------


In [54]:
sentiment_polarity=[textblob.TextBlob(Reviews).sentiment.polarity for Reviews in reviews]
predicted_sentiments=['positive'if score >= 0.1 else 'negative' for score in sentiment_polarity]

In [62]:
sentiment_polarity

[0.2816666666666667,
 0.6716666666666666,
 0.10200000000000001,
 0.2816666666666667,
 0.6716666666666666,
 0.10200000000000001,
 0.2816666666666667,
 0.6716666666666666,
 0.10200000000000001,
 0.2816666666666667,
 0.6716666666666666,
 0.10200000000000001,
 0.4030555555555555,
 0.08333333333333334,
 0.3375,
 0.02499999999999999,
 0.0,
 0.3,
 0.7,
 0.9,
 1.0,
 0.31666666666666665,
 0.4030555555555555,
 0.08333333333333334,
 0.3375,
 0.02499999999999999,
 0.0,
 0.3,
 0.7,
 0.9,
 1.0,
 0.31666666666666665,
 0.0,
 1.0,
 0.7,
 0.7,
 0.09999999999999998,
 0.6,
 0.7,
 0.0,
 0.7,
 0.7,
 0.06298185941043083,
 0.22766666666666668,
 0.29625,
 0.13750000000000004,
 0.3333333333333333,
 0.4030555555555555,
 0.08333333333333334,
 0.3375,
 0.02499999999999999,
 0.0,
 0.3,
 0.7,
 0.9,
 1.0,
 0.31666666666666665,
 0.4030555555555555,
 0.08333333333333334,
 0.3375,
 0.02499999999999999,
 0.0,
 0.3,
 0.7,
 0.9,
 1.0,
 0.31666666666666665,
 0.0,
 1.0,
 0.7,
 0.7,
 0.09999999999999998,
 0.6,
 0.7,
 0.0,
 0.

In [64]:
predicted_sentiments

['positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'positive',
 'negative',
 'positive',
 'positive',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'positive',
 'negative',
 'positive',
 'positive',

In [68]:
#create a new data frame
df = pd.DataFrame({'sentiments':predicted_sentiments})
print(df)

    sentiments
0     positive
1     positive
2     positive
3     positive
4     positive
..         ...
527   negative
528   negative
529   negative
530   positive
531   positive

[532 rows x 1 columns]


In [57]:
import pandas as pd

In [71]:
dataset_ = pd.concat([sentiment_,df], axis=1)

In [73]:
dataset_

,sentence,review_no,reviews,score,Sentiment,sentiments
0,0,0,i using toshiba pen drive,0.000000,positive,positive
1,1,0,this iiird puchasing amazon,0.000000,positive,positive
2,2,0,this device easy use,0.433333,positive,positive
3,0,1,very good memory card easy use,0.671667,positive,positive
4,0,2,highly reliable japanese quality product,0.080000,positive,positive
...,...,...,...,...,...,...
905,0,529,damage products dai deaaaa returned date bhi k...,0.000000,positive,NaN
906,0,530,it took 115 minutes copy 115gb movie,0.000000,positive,NaN
907,1,530,received product shown i amage,0.000000,positive,NaN
908,2,530,it genuine product,0.400000,positive,NaN


##Supervised learning

In [75]:

from bs4 import BeautifulSoup
import numpy as np
import re
import tqdm
import unicodedata


def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    [s.extract() for s in soup(['iframe', 'script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    return stripped_text

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text
import re
contractions_dict = {
    'didn\'t': 'did not',
    'don\'t': 'do not',
    "aren't": "are not",
    "can't": "cannot",
    "cant": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "didnt": "did not",
    "doesn't": "does not",
    "doesnt": "does not",
    "don't": "do not",
    "dont" : "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he had",
    "he'd've": "he would have",
    "he'll": "he will",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i had",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'm": "i am",
    "im": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'll": "it will",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she had",
    "she'd've": "she would have",
    "she'll": "she will",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "that's": "that is",
    "there's": "there is",
    "they'd": "they had",
    "they'd've": "they would have",
    "they'll": "they will",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we had",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who's": "who is",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "you'll": "you will",
    "you're": "you are",
    "you've": "you have"
    }

contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))

def expand_contractions(s, contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)


def pre_process_corpus(docs):
    norm_docs = []
    for doc in tqdm.tqdm(docs):
        doc = strip_html_tags(doc)
        doc = doc.translate(doc.maketrans("\n\t\r", "   "))
        doc = doc.lower()
        doc = remove_accented_chars(doc)
        doc = expand_contractions(doc)
        # lower case and remove special characters\whitespaces
        doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
        doc = re.sub(' +', ' ', doc)
        doc = doc.strip()  
        norm_docs.append(doc)
  
    return norm_docs


import tqdm
def corpus_pre_processor(corpus):
    norm_corpus = []
    for doc in tqdm.tqdm(corpus):
        norm_corpus.append(normalize_document(doc))
    return norm_corpus
def normalize_corpus(doc):
    #Lowercase, remove special char \whitespace
    #remove stopwords
    #expand contraction
    words= word_tokenize(doc)
    doc = " ".join([word.lower() for word in words if word not in stop])
    doc = re.sub(r'[^a-zA-Z0-9\s]','',doc,re.I|re.A)
    doc =doc.strip()
    doc = expand_contractions(doc)
    return doc


In [77]:
#dividing ito x and y
reviews= dataset_['reviews'].values
sentiments = dataset_['Sentiment'].values

In [80]:
train_reviews=reviews[:60]
train_sentiment = sentiments[:60]

test_reviews = reviews[60:]
test_sentiment = sentiments[60:]

In [82]:
%%time
norm_train_reviews = pre_process_corpus(train_reviews)
norm_test_reviews = pre_process_corpus(test_reviews)

100%|██████████████████████████████████████████████████████████████████████████████| 850/850 [00:00<00:00, 4897.24it/s]

Wall time: 192 ms


In [83]:
%%time
from sklearn.feature_extraction.text import CountVectorizer

#biuld BOW
cv = CountVectorizer(binary = False, min_df = 5, max_df = 1.0, ngram_range=(1,2))

cv_train_features = cv.fit_transform(norm_train_reviews)

Wall time: 185 ms


In [84]:
cv_test_features = cv.transform(norm_test_reviews)

Logistic Regression

In [85]:
%%time
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(penalty='l2',max_iter=500,C=1,solver='lbfgs')
lr.fit (cv_train_features,train_sentiment)

Wall time: 481 ms


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [86]:
lr_predictions = lr.predict(cv_test_features)

In [87]:
from sklearn.metrics import confusion_matrix,classification_report
print(classification_report(test_sentiment,lr_predictions))
print(confusion_matrix(test_sentiment,lr_predictions))

              precision    recall  f1-score   support

    negative       0.60      0.05      0.09       124
    positive       0.86      0.99      0.92       726

    accuracy                           0.86       850
   macro avg       0.73      0.52      0.51       850
weighted avg       0.82      0.86      0.80       850

[[  6 118]
 [  4 722]]


In [88]:
from sklearn.metrics import accuracy_score
accuracy_score(test_sentiment,lr_predictions)

0.8564705882352941

Random Forest

In [110]:
#dividing ito train and test
review_R= dataset_['reviews'].values
sentiment_R = dataset_['Sentiment'].values

In [124]:
train_review_R=review_R[:60]
train_sentiment_R = sentiment_R[:60]

test_review_R = review_R[60:]
test_sentiment_R = sentiment_R[60:]

In [113]:
%%time
norm_train_review_R = pre_process_corpus(train_review_R)
norm_test_review_R = pre_process_corpus(test_review_R)

100%|██████████████████████████████████████████████████████████████████████████████| 850/850 [00:00<00:00, 6710.82it/s]

Wall time: 148 ms


In [114]:
%%time
from sklearn.feature_extraction.text import CountVectorizer

#biuld BOW
cv = CountVectorizer(binary = False, min_df = 5, max_df = 1.0, ngram_range=(1,2))

cv_train_features_R = cv.fit_transform(norm_train_review_R)

Wall time: 4.99 ms


In [115]:
cv_test_features_R = cv.transform(norm_test_review_R)

In [118]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

lr_R = RandomForestClassifier(n_estimators=10,
                                    criterion='gini')


In [120]:
lr_R.fit(cv_train_features_R,train_review_R)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [121]:
lr_predictions_R = lr.predict(cv_test_features_R)

In [122]:
lr_predictions_R

array(['positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'negative',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'negative', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positive',
       'positive', 'positive', 'positive', 'positive', 'positi

In [126]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(test_sentiment_R,lr_predictions_R))
print(confusion_matrix(test_sentiment_R,lr_predictions_R))

              precision    recall  f1-score   support

    negative       0.60      0.05      0.09       124
    positive       0.86      0.99      0.92       726

    accuracy                           0.86       850
   macro avg       0.73      0.52      0.51       850
weighted avg       0.82      0.86      0.80       850

[[  6 118]
 [  4 722]]


In [127]:
from sklearn.metrics import accuracy_score
accuracy_score(test_sentiment_R,lr_predictions_R)

0.8564705882352941